# Results summary

In [61]:
import pandas as pd
from glob import glob
import os
import torch
from experiments.epfl_utils import label_map
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score

# Load val dataframe

In [2]:
val_df_path = "val.json"

In [3]:
val_df = pd.read_json(val_df_path)
val_df.head()

,rgb,infrared,label
100,/datasets2/users/pmazurek/nirscene1/indoor/004...,/datasets2/users/pmazurek/nirscene1/indoor/004...,indoor
103,/datasets2/users/pmazurek/nirscene1/indoor/005...,/datasets2/users/pmazurek/nirscene1/indoor/005...,indoor
105,/datasets2/users/pmazurek/nirscene1/indoor/005...,/datasets2/users/pmazurek/nirscene1/indoor/005...,indoor
116,/datasets2/users/pmazurek/nirscene1/street/001...,/datasets2/users/pmazurek/nirscene1/street/001...,street
120,/datasets2/users/pmazurek/nirscene1/street/001...,/datasets2/users/pmazurek/nirscene1/street/001...,street


In [68]:
val_df.label.value_counts()

urban          12
mountain       11
indoor         11
country        11
forest         11
field          10
oldbuilding    10
street         10
water          10
Name: label, dtype: int64

In [50]:
true_labels = val_df.label.map(label_map)

# Load saved predictions

In [4]:
rootdir = os.path.join("results")

prediction_files = [] 

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".pt"):
            prediction_files.append(filepath)

In [5]:
f"{len(prediction_files)} prediction files"

'48 prediction files'

In [11]:
for folder in set([os.path.basename(os.path.dirname(path)) for path in prediction_files]):
    print(f"{folder}: {len([path for path in prediction_files if folder in path])} files")

rgb_only: 12 files
rgb_infrared_combined: 12 files
infrared_only: 12 files
rgb_infrared_as_two_separate_inputs: 12 files


In [17]:
set([os.path.basename(path).replace(".pt", "") for path in prediction_files])

{'densenet_121_from_scratch',
 'densenet_121_pretrained',
 'densenet_169_from_scratch',
 'densenet_169_pretrained',
 'efficientnet_b0_from_scratch',
 'efficientnet_b0_pretrained',
 'efficientnet_b2_from_scratch',
 'efficientnet_b2_pretrained',
 'resnet_18_from_scratch',
 'resnet_18_pretrained',
 'resnet_50_from_scratch',
 'resnet_50_pretrained'}

# Compare obtained results

In [ ]:
for folder in set([os.path.basename(os.path.dirname(path)) for path in prediction_files]):
    print(f"{folder}: {len([path for path in prediction_files if folder in path])} files")

In [70]:
df = pd.DataFrame(columns=["path", "network_name", "experiment_type", "accuracy", "recall", "precission", "f1"])

for path in prediction_files:
    predictions = torch.load(path).argmax(axis=-1)
    
    network_name = os.path.basename(path)
    experiment_type = os.path.basename(os.path.dirname(path))
    accuracy = accuracy_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions, average="macro")
    precission = precision_score(true_labels, predictions, average="macro")
    f1 = f1_score(true_labels, predictions, average="macro")
    record = [path, network_name, experiment_type, accuracy, recall, precission, f1]
    
    df.loc[len(df)] = record
    
df.head()

,path,network_name,experiment_type,accuracy,recall,precission,f1
0,results/rgb_infrared_combined/resnet_50_from_s...,resnet_50_from_scratch.pt,rgb_infrared_combined,0.625000,0.627609,0.638811,0.619346
1,results/rgb_infrared_combined/resnet_50_pretra...,resnet_50_pretrained.pt,rgb_infrared_combined,0.760417,0.759596,0.782597,0.757354
2,results/rgb_infrared_combined/densenet_121_pre...,densenet_121_pretrained.pt,rgb_infrared_combined,0.781250,0.777273,0.802549,0.774515
3,results/rgb_infrared_combined/efficientnet_b0_...,efficientnet_b0_pretrained.pt,rgb_infrared_combined,0.843750,0.845455,0.854856,0.840511
4,results/rgb_infrared_combined/densenet_121_fro...,densenet_121_from_scratch.pt,rgb_infrared_combined,0.666667,0.668182,0.685558,0.660960


## Overview

In [84]:
df[df.accuracy == df.accuracy.max()]

,path,network_name,experiment_type,accuracy,recall,precission,f1
46,results/rgb_only/efficientnet_b2_pretrained.pt,efficientnet_b2_pretrained.pt,rgb_only,0.895833,0.896465,0.901235,0.894493


In [85]:
df[df.recall == df.recall.max()]

,path,network_name,experiment_type,accuracy,recall,precission,f1
46,results/rgb_only/efficientnet_b2_pretrained.pt,efficientnet_b2_pretrained.pt,rgb_only,0.895833,0.896465,0.901235,0.894493


In [86]:
df[df.precission == df.precission.max()]

,path,network_name,experiment_type,accuracy,recall,precission,f1
46,results/rgb_only/efficientnet_b2_pretrained.pt,efficientnet_b2_pretrained.pt,rgb_only,0.895833,0.896465,0.901235,0.894493


In [74]:
df[df.f1 == df.f1.max()] 

,path,network_name,experiment_type,accuracy,recall,precission,f1
46,results/rgb_only/efficientnet_b2_pretrained.pt,efficientnet_b2_pretrained.pt,rgb_only,0.895833,0.896465,0.901235,0.894493


## Resnet

In [83]:
resnet_df = df[df.network_name.map(lambda name: "resnet" in name)]
resnet_df

,path,network_name,experiment_type,accuracy,recall,precission,f1
0,results/rgb_infrared_combined/resnet_50_from_s...,resnet_50_from_scratch.pt,rgb_infrared_combined,0.625000,0.627609,0.638811,0.619346
1,results/rgb_infrared_combined/resnet_50_pretra...,resnet_50_pretrained.pt,rgb_infrared_combined,0.760417,0.759596,0.782597,0.757354
5,results/rgb_infrared_combined/resnet_18_pretra...,resnet_18_pretrained.pt,rgb_infrared_combined,0.739583,0.739226,0.759042,0.730895
6,results/rgb_infrared_combined/resnet_18_from_s...,resnet_18_from_scratch.pt,rgb_infrared_combined,0.614583,0.617340,0.642388,0.611630
12,results/infrared_only/resnet_50_from_scratch.pt,resnet_50_from_scratch.pt,infrared_only,0.562500,0.560438,0.566479,0.549463
13,results/infrared_only/resnet_50_pretrained.pt,resnet_50_pretrained.pt,infrared_only,0.729167,0.728956,0.726491,0.708748
17,results/infrared_only/resnet_18_pretrained.pt,resnet_18_pretrained.pt,infrared_only,0.802083,0.804040,0.831186,0.803599
18,results/infrared_only/resnet_18_from_scratch.pt,resnet_18_from_scratch.pt,infrared_only,0.593750,0.590741,0.586924,0.570492
24,results/rgb_infrared_as_two_separate_inputs/re...,resnet_50_from_scratch.pt,rgb_infrared_as_two_separate_inputs,0.656250,0.656061,0.712735,0.646303
25,results/rgb_infrared_as_two_separate_inputs/re...,resnet_50_pretrained.pt,rgb_infrared_as_two_separate_inputs,0.822917,0.822391,0.832051,0.815071


In [87]:
resnet_df[resnet_df.accuracy == resnet_df.accuracy.max()]

,path,network_name,experiment_type,accuracy,recall,precission,f1
37,results/rgb_only/resnet_50_pretrained.pt,resnet_50_pretrained.pt,rgb_only,0.84375,0.844613,0.85303,0.840386


In [88]:
resnet_df[resnet_df.recall == resnet_df.recall.max()]

,path,network_name,experiment_type,accuracy,recall,precission,f1
37,results/rgb_only/resnet_50_pretrained.pt,resnet_50_pretrained.pt,rgb_only,0.84375,0.844613,0.85303,0.840386


In [89]:
resnet_df[resnet_df.precission == resnet_df.precission.max()]

,path,network_name,experiment_type,accuracy,recall,precission,f1
37,results/rgb_only/resnet_50_pretrained.pt,resnet_50_pretrained.pt,rgb_only,0.84375,0.844613,0.85303,0.840386


In [90]:
resnet_df[resnet_df.f1 == resnet_df.f1.max()]

,path,network_name,experiment_type,accuracy,recall,precission,f1
37,results/rgb_only/resnet_50_pretrained.pt,resnet_50_pretrained.pt,rgb_only,0.84375,0.844613,0.85303,0.840386


## Desnenet

In [96]:
densenet_df = df[df.network_name.map(lambda name: "densenet" in name)]
densenet_df

,path,network_name,experiment_type,accuracy,recall,precission,f1
2,results/rgb_infrared_combined/densenet_121_pre...,densenet_121_pretrained.pt,rgb_infrared_combined,0.781250,0.777273,0.802549,0.774515
4,results/rgb_infrared_combined/densenet_121_fro...,densenet_121_from_scratch.pt,rgb_infrared_combined,0.666667,0.668182,0.685558,0.660960
7,results/rgb_infrared_combined/densenet_169_pre...,densenet_169_pretrained.pt,rgb_infrared_combined,0.781250,0.779966,0.800044,0.775668
11,results/rgb_infrared_combined/densenet_169_fro...,densenet_169_from_scratch.pt,rgb_infrared_combined,0.666667,0.666162,0.690960,0.650519
14,results/infrared_only/densenet_121_pretrained.pt,densenet_121_pretrained.pt,infrared_only,0.802083,0.802020,0.817811,0.795958
16,results/infrared_only/densenet_121_from_scratc...,densenet_121_from_scratch.pt,infrared_only,0.635417,0.629461,0.626388,0.607556
19,results/infrared_only/densenet_169_pretrained.pt,densenet_169_pretrained.pt,infrared_only,0.802083,0.801178,0.825807,0.798976
23,results/infrared_only/densenet_169_from_scratc...,densenet_169_from_scratch.pt,infrared_only,0.583333,0.580471,0.596169,0.563722
26,results/rgb_infrared_as_two_separate_inputs/de...,densenet_121_pretrained.pt,rgb_infrared_as_two_separate_inputs,0.802083,0.806734,0.827804,0.791011
28,results/rgb_infrared_as_two_separate_inputs/de...,densenet_121_from_scratch.pt,rgb_infrared_as_two_separate_inputs,0.708333,0.706229,0.728058,0.707180


In [91]:
densenet_df[densenet_df.accuracy == densenet_df.accuracy.max()]

,path,network_name,experiment_type,accuracy,recall,precission,f1
43,results/rgb_only/densenet_169_pretrained.pt,densenet_169_pretrained.pt,rgb_only,0.864583,0.862121,0.868652,0.857901


In [92]:
densenet_df[densenet_df.recall == densenet_df.recall.max()]

,path,network_name,experiment_type,accuracy,recall,precission,f1
43,results/rgb_only/densenet_169_pretrained.pt,densenet_169_pretrained.pt,rgb_only,0.864583,0.862121,0.868652,0.857901


In [93]:
densenet_df[densenet_df.precission == densenet_df.precission.max()]

,path,network_name,experiment_type,accuracy,recall,precission,f1
43,results/rgb_only/densenet_169_pretrained.pt,densenet_169_pretrained.pt,rgb_only,0.864583,0.862121,0.868652,0.857901


In [94]:
densenet_df[densenet_df.f1 == densenet_df.f1.max()]

,path,network_name,experiment_type,accuracy,recall,precission,f1
43,results/rgb_only/densenet_169_pretrained.pt,densenet_169_pretrained.pt,rgb_only,0.864583,0.862121,0.868652,0.857901


## Efficientnet

In [95]:
efficientnet_df = df[df.network_name.map(lambda name: "efficientnet" in name)]
efficientnet_df

,path,network_name,experiment_type,accuracy,recall,precission,f1
3,results/rgb_infrared_combined/efficientnet_b0_...,efficientnet_b0_pretrained.pt,rgb_infrared_combined,0.843750,0.845455,0.854856,0.840511
8,results/rgb_infrared_combined/efficientnet_b0_...,efficientnet_b0_from_scratch.pt,rgb_infrared_combined,0.114583,0.111111,0.012731,0.022845
9,results/rgb_infrared_combined/efficientnet_b2_...,efficientnet_b2_from_scratch.pt,rgb_infrared_combined,0.114583,0.111111,0.012731,0.022845
10,results/rgb_infrared_combined/efficientnet_b2_...,efficientnet_b2_pretrained.pt,rgb_infrared_combined,0.875000,0.874074,0.879202,0.868453
15,results/infrared_only/efficientnet_b0_pretrain...,efficientnet_b0_pretrained.pt,infrared_only,0.812500,0.812458,0.830951,0.813665
20,results/infrared_only/efficientnet_b0_from_scr...,efficientnet_b0_from_scratch.pt,infrared_only,0.104167,0.111111,0.011574,0.020964
21,results/infrared_only/efficientnet_b2_from_scr...,efficientnet_b2_from_scratch.pt,infrared_only,0.114583,0.111111,0.012731,0.022845
22,results/infrared_only/efficientnet_b2_pretrain...,efficientnet_b2_pretrained.pt,infrared_only,0.854167,0.856734,0.870282,0.855172
27,results/rgb_infrared_as_two_separate_inputs/ef...,efficientnet_b0_pretrained.pt,rgb_infrared_as_two_separate_inputs,0.885417,0.885354,0.894573,0.882108
32,results/rgb_infrared_as_two_separate_inputs/ef...,efficientnet_b0_from_scratch.pt,rgb_infrared_as_two_separate_inputs,0.104167,0.111111,0.011574,0.020964


In [97]:
efficientnet_df[efficientnet_df.accuracy == efficientnet_df.accuracy.max()]

,path,network_name,experiment_type,accuracy,recall,precission,f1
46,results/rgb_only/efficientnet_b2_pretrained.pt,efficientnet_b2_pretrained.pt,rgb_only,0.895833,0.896465,0.901235,0.894493


In [98]:
efficientnet_df[efficientnet_df.recall == efficientnet_df.recall.max()]

,path,network_name,experiment_type,accuracy,recall,precission,f1
46,results/rgb_only/efficientnet_b2_pretrained.pt,efficientnet_b2_pretrained.pt,rgb_only,0.895833,0.896465,0.901235,0.894493


In [100]:
efficientnet_df[efficientnet_df.precission == efficientnet_df.precission.max()]

,path,network_name,experiment_type,accuracy,recall,precission,f1
46,results/rgb_only/efficientnet_b2_pretrained.pt,efficientnet_b2_pretrained.pt,rgb_only,0.895833,0.896465,0.901235,0.894493


In [101]:
efficientnet_df[efficientnet_df.f1 == efficientnet_df.f1.max()]

,path,network_name,experiment_type,accuracy,recall,precission,f1
46,results/rgb_only/efficientnet_b2_pretrained.pt,efficientnet_b2_pretrained.pt,rgb_only,0.895833,0.896465,0.901235,0.894493
